In [1]:
!pip install virtualenv --quiet
!python -m virtualenv .

!source ./bin/activate; pip install git+https://github.com/huggingface/diffusers.git --quiet
!source ./bin/activate; pip install transformers==v4.27.1 scipy ftfy accelerate --quiet

import sys
sys.path.append("./lib/python3.9/site-packages")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 36.1 MB/s eta 0:00:00
created virtual environment CPython3.9.16.final.0-64 in 693ms
  creator CPython3Posix(dest=/content, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.0.1, setuptools==67.4.0, wheel==0.38.4
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.7/763.7 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 71.9 MB/s eta 0:00:

In [2]:
#@title Setup Diffusion module

from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch

repo_id = "andite/pastel-mix" #@param ["CompVis/stable-diffusion-v1-4", "stabilityai/stable-diffusion-2-base", "hakurei/waifu-diffusion", "andite/anything-v4.0", "prompthero/openjourney-v2", "andite/pastel-mix", "gsdf/Counterfeit-V2.5"]
device = "cuda" #@param { type: "string" }
pipe = DiffusionPipeline.from_pretrained(
    repo_id, 
    # torch_dtype=torch.float16, 
    # revision="fp16"
    )

# pipe.scheduler = DPMSolverMultistepScheduler.from_config(
#     pipe.scheduler.config
#     )
pipe = pipe.to(device)

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
#@title Genrate Image

seed=0 #@param {type: "integer"}
#@markdown Set `0` to randomizeing
prompt="" #@param { type: "string"}
num_inference_steps=50 #@param { type: "integer"} 
#@markdown `50` is default.
height=800 #@param { type: "integer"}
width=400 #@param { type: "integer"}
image=[]

if seed==0:
  image = pipe(
      prompt, 
      num_inference_steps=num_inference_steps,
      height=height, 
      width=width
    ).images[0]
else:
  generator = torch.Generator(device=device).manual_seed(seed)
  image = pipe(
      prompt, 
      generator=generator,
      guidance_scale=7.5,
      height=height, 
      width=width
  ).images[0]

image

In [ ]:
#@title Clear PyTourch cahe
torch.cuda.empty_cache()

In [ ]:
# !git clone https://github.com/huggingface/diffusers.git
# !wget https://civitai.com/api/download/models/19084 -o diffusers.safetensors
!pip install safetensors --quiet
!python diffusers/scripts/convert_lora_safetensor_to_diffusers.py --base_model_path stabilityai/stable-diffusion-2-base --checkpoint_path diffusers.safetensors --dump_path ./output